# TV Script Generation
This project will utilize RNN's to generate plausible TV scripts. Simpson's scripts from 27 seasons will be used as input. Specifically, those scenes that take place at Moe's Tavern.

In [4]:
import helper
import numpy as np

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
text = text[81:]

In [5]:
# Exploring the data

view_sentence_range = (0, 10)

print("Dataset Stats")
print("Number of unique words (Roughly): {}".format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print("Number of scenes: {}".format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene" {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print("Number of lines: {}".format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print("Average number of words in each line: {}".format(np.average(word_count_sentence)))

print()
print("The sentences {} to {}:".format(*view_sentence_range))
print("\n".join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))


Dataset Stats
Number of unique words (Roughly): 11492
Number of scenes: 262
Average number of sentences in each scene" 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implementing Preprocessing Functions

### Lookup Table
To create a word embedding, first transform the words to ids. Following function will return a tuple containing two dictionaries: One to translate words to id's, and another to translate id's to words.

In [7]:
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    counts = Counter(text)
    vocab = sorted(counts, key=counts.get, reverse=True)
    vocab_to_int = {word: ii for ii, word in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))
    
    return (vocab_to_int, int_to_vocab)


### Tokenize Punctuation
We need to split the script into a word array using spaces as delimiters. The punctuation used in the script would make it difficult for the network to differentiate between words like "hello!" and "hello?". The following function will tokenize the punctuation using a lookup table which will later be used to replace punctuation in the script so that they are interpreted individually instead of as a part of another word. 

In [8]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    tokens = {
        "."  : "||Period||",
        ","  : "||Comma||",
        "\"" : "||Quotation_Mark||",
        ";"  : "||Semicolon||",
        "!"  : "||Exclamation_Mark||",
        "?"  : "||Question_Mark||",
        "("  : "||Left_Parentheses||",
        ")"  : "||Right_Parentheses||",
        "--" : "||Dash||",
        "\n" : "||Return||"
    }
    
    return tokens

## Preprocess the data and save
Utilizing the prepreocessing functions from the helper function.

In [9]:
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

## Load preprocessed data
The follownig code block will reload the preprocessed data so that all the above code does not need to be re-run each time.

In [10]:
import numpy as np

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Building Neural Net Components

### Input
following function creates placeholders for the neural network. 

In [11]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    
    return inputs, targets, learning_rate


### Build RNN Cell 
Stack tensorflow basicLSTM cells into a MultiRNNCell. 

In [12]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    
    lstm_layers = 3
    
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.MultiRNNCell([lstm] * lstm_layers)
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, 'initial_state')
    
    return cell, initial_state


### Word Embedding
Apply embedding to input data using tensorflow. Returns the embedded sequence.


In [14]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    embed = tf.contrib.layers.embed_sequence(input_data, 
                                             vocab_size, 
                                             embed_dim)
    
    return embed

### Build RNN
Utilize the rnn cells function to create the RNN. 

In [15]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, name="final_state")
    
    return outputs, final_state

### Build the Neural Network
Apply the above functions to build the network

In [16]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    embedded = get_embed(input_data, 
                         vocab_size, 
                         embed_dim)
    
    out, final_state = build_rnn(cell, embedded)
    
    logits = tf.contribe.layers.fully_connected(out, 
                                                vocab_size, 
                                                activation_fn=None)
    
    return logits, final_state

## Batches

Following function will create batches of input and targets using int_text. The batches will be Numpy arrays with the shape (num_of_batches, 2, batch_size, seq_length). Each batch contains two elements:

* A single batch of input with shape [batch_size, sequence_length]
* A single batch of tarets with shape [batch_Size, sequence_length]

If the last batch cannont fill the batch size with enough data, it will be dropped. 
